In [77]:

import warnings

import numpy as np
import pandas as pd
from scipy import stats

warnings.filterwarnings('ignore')


%precision 3
np.random.seed(1111)

# 統計的仮説検定

In [29]:
# 標本 Data: 14日間のフライドポテトの重さを準備
df = pd.read_csv('../data/ch11_potato.csv')
sample = np.array(df['重さ'])
display(sample)

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [30]:
# 標本平均を求める
s_mean = np.mean(sample)
s_mean

128.451

## 統計的仮説検定とは（statistical hypothesis testing）

### 統計的仮説検定の基本
- フライドポテトの母平均が130gより少ないかどうかを確かめたい。
- 前提として
  - 母集団が正規分布に従っている
  - 母分散は９
とわかっている。

母平均が「130g」という仮定をすると、標本平均は、N(130, 9)に従うことになる

In [31]:
# 標本平均が P(標本平均<=x)=0.05 を満たす x を考える
rv = stats.norm(130, np.sqrt(9 / 14))
rv.isf(0.95)

128.681

標本平均が 126.681g 以下の重さになることは５％の確率でしか発生しない = 標本平均が 128.451g となったのは５％の確率でしか発生しない珍事な出来事といえる。
よって「母平均が 130g より少ない」と考え結論づける。

In [32]:
# 帰無仮説「母平均 130g」
# 検定統計量に標本平均を標準化した Z を使用する。
# 検定統計量が臨界値より小さければ帰無仮説を棄却し、そうでないときに帰無仮説を採択する。

z = (s_mean - 130) / np.sqrt(9/14)
print(f'統計検定量: {z:.3f}')

統計検定量: -1.932


In [33]:
rv = stats.norm()

print(f'臨界値: {rv.isf(0.95):.3f}')

臨界値: -1.645


検定統計量と臨界値を比較すると検定統計量のほうが小さい値となっている。
= 帰無仮説は棄却
= 平均は、130g より少ないという結論になる

In [34]:
# ｐ値を使った仮説検定について確認する。

# 検定統計量（標本平均を標準化した Z）からｐ値を求める
print(f'ｐ値: {rv.cdf(z):.3f}')

ｐ値: 0.027


ｐ値が有意水準 0.05 より小さい値になった。
= 帰無仮説は棄却

### 片側検定と両側検定
フライドポテトの「母平均は 130g ではない」という対立仮説で仮説検定を行なう。（※130g より少ない、多いの両側検定）

In [35]:
# 検定統計量を求める
z = (s_mean - 130) / np.sqrt(9/14)
print(f'検定統計量: {z:.3f}（※標本平均を標準化した値）')

検定統計量: -1.932（※標本平均を標準化した値）


In [36]:
# 臨界値を求める
# 両側検定のため標準正規分布の95%区間によって求める
rv = stats.norm()
rv.interval(0.95)

(-1.960, 1.960)

臨界値と統計検定量を比較してみると、統計検定量が採択域にはいっているため帰無仮説は棄却されない。
※両側検定に比べると片側検定は棄却しやすい。

In [37]:
# ｐ値を求める
# 両側検定のｐ値は、上側と下側の両方の面積を考慮し、累積密度関数の値を２倍にすることによって求められる。
print(f'ｐ値（両側検定: {rv.cdf(z) * 2:.3f}')

ｐ値（両側検定: 0.053


ｐ値が有意水準 0.05 より大きくなっていることからも帰無仮説は棄却されない。

### 仮説検定における２つの過誤

#### 第一種の過誤
帰無仮説が正しいときに、帰無仮説を棄却してしまう過誤がどのくらいの割合で発生してしまうのか Simulation してみる。

In [38]:
# 母集団の確率分布は N(130, 9)
rv = stats.norm(130, 3)

In [39]:
# この母集団から 14個の標本を抽出して仮説検定を行なう、という作業を10000回行なう
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9 / 14)
    if z < c:
        cnt += 1

print(f'第一種の過誤を犯す割合（「平均130g」であるにも関わらず「平均は 130g より少ない」と結論づけてしまう割合）: {cnt / n_samples:.3f}')

第一種の過誤を犯す割合（「平均130g」であるにも関わらず「平均は 130g より少ない」と結論づけてしまう割合）: 0.053


#### 第二種の過誤
対立仮説が正しいときに、帰無仮説を採択してしまう過誤（「母平均が 130g より少ない」のに「母平均が 130g より少ない」という結論が得ることができない）がどのくらいの割合で発生するか simulation してみる。

In [40]:
# 母集団のへの確率分布は N(128, 9)
rv = stats.norm(128, 3)

In [41]:
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9 / 14)
    if z >= c:
        cnt += 1

print(f'第二種の過誤を犯してしまう割合; {cnt / n_samples:.3f}')

第二種の過誤を犯してしまう割合; 0.197


## 基本的な仮説検定

### 正規分布の母平均の検定（母分散既知）

In [42]:
# 正規分布の母平均の仮説検定（母分散既知）を関数として実装

def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1 - alpha)

    z = (s_mean - mean0) / np.sqrt(p_var / n)
    if interval[0] <= z <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')
    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1 - rv.cdf(z)) * 2
    print(f'ｐ値は{p:.3f}')

In [43]:
# フライドポテトの標本Data で実行
pmean_test(sample, 130, 9)

帰無仮説を採択
ｐ値は0.053


### 正規分布の母分散の検定

In [44]:
# 正規分布の母分散の検定を関数として実装
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n - 1)
    interval = rv.interval(1 - alpha)

    y = (n - 1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')
    if y < rv.isf(0.5):
        p = rv.cdf(y) * 2
    else:
        p = (1 - rv.cdf(y)) * 2
    print(f'ｐ値は{p:.3f}')

In [45]:
# フライドポテトの標本Data で実行
pvar_test(sample, 9)

帰無仮説を採択
ｐ値は0.085


### 正規分布の母平均の検定（母分散未知）

In [46]:
# 正規分布の母平均の検定（母分散未知）を関数として実装
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n - 1)
    interval = rv.interval(1 - alpha)

    t = (s_mean - mean0) / np.sqrt(u_var / n)
    if interval[0] <= t <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')
    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'ｐ値は{p:.3f}')

In [47]:
# フライドポテトの標本に対して実行
pmean_test(sample, 130)

帰無仮説を採択
ｐ値は0.169


In [48]:
# `scipy.stats` の `ttest_1samp()`関数を利用してみる
t, p = stats.ttest_1samp(sample, 130)
print(f'統計検定量: {t:.3f}', f'ｐ値: {p:.3f}')

統計検定量: -1.455 ｐ値: 0.169


## ２標本問題（two-sample problem）に関する仮説検定

### 対応のある t検定（paired t-test)
Data に対応があり、差に正規分布を仮定できる場合の平均値差の検定

In [49]:
# Data の準備
# 20人に１週間筋トレをしてもらい、その前後で集中力を測った Test結果
training_rel = pd.read_csv('../data/ch11_training_rel.csv')
print(training_rel.shape)
training_rel.head()

(20, 2)


,前,後
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


筋トレに集中力を向上させる効果があるかどうかを、筋トレ前と筋トレ御の集中力Test の平均点を比較することで判断する。
そのために次の仮説検定を行なう
- 帰無仮説: 筋トレ前の平均点 - 筋トレ後の平均的 = 0
- 対立仮説: 筋トレ前の平均点 - 筋トレ後の平均的 = 0 ではない

In [50]:
# 個々の Data の差を考える
training_rel['差'] = training_rel['前'] - training_rel['後']
training_rel.head()

,前,後,差
0,59,41,18
1,52,63,-11
2,55,68,-13
3,61,59,2
4,59,84,-25


仮説検定は差の平均として
- 帰無仮説: 平均点の差 = 0
- 対立仮説: 平均点の差 = 0 ではない

と言い換えることができる。
そのことから１標本のｔ検定（正規分布の母平均の検定（母分散未知））に帰着し、仮説検定を行なう。

In [51]:
# １標本のｔ検定を行なう
t, p = stats.ttest_1samp(training_rel['差'], 0)
print(f'ｐ値: {p:.3f}')

ｐ値: 0.040


ｐ値が有意水準を下回った為、帰無仮説を棄却する。
筋トレは集中力に有意な差をもたらす様子。

In [52]:
# `ttest_rel`関数を利用して仮説検定を行なう
t, p = stats.ttest_rel(training_rel['後'], training_rel['前'])
print(f'ｐ値: {p:.3f}')

ｐ値: 0.040


先ほどと同様のｐ値が確認できた。

### 対応のないｔ検定（independent t-test）
Data に対応がなく、２標本の母集団に正規分布を仮定できる場合の平均値の差の検定

In [53]:
# Data の準備
training_ind = pd.read_csv('../data/ch11_training_ind.csv')
print(training_ind.shape)
training_ind.head()

(20, 2)


,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


筋トレに集中力を向上させる効果があるかどうかを
- A: 普段から筋トレをしていない集団
- B: 普段から筋トレをしている集団

の集中力テストの平均点を比較することで判断する。

そのために次の仮説検定を行なう。
- 帰無仮説: Aの平均点 - Bの平均点 = 0
- 対立仮説: Aの平均点 - Bの平均点 = 0 ではない

In [54]:
# `stats.ttest_ind`関数を使用。
# 引数`equal_var = False` を指定することでウェルチの方法で計算される
t, p = stats.ttest_ind(training_ind['A'], training_ind['B'], equal_var=False)

print(f'ｐ値: {p:.3f}')

ｐ値: 0.087


ｐ値が有意水準よりも大きいため、帰無仮説は採択され A と B の間には平均点に有意な差があるとはいえない、という結論になった。

### ウィルコクソンの符号付き順位検定（Wilcoxon signed-rank test）

In [55]:
# Data の準備
training_rel = pd.read_csv('../data/ch11_training_rel.csv')
toy_df = training_rel[:6].copy()
toy_df

,前,後
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84
5,45,37


In [56]:
# 対応のある Data なので、Data の差に着目する
diff = toy_df['後'] - toy_df['前']
toy_df['差'] = diff
toy_df

,前,後,差
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [57]:
# 差の絶対値が小さいものから順に順位をつける
# `stats.rankdata'関数を利用
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['順位'] = rank
toy_df

,前,後,差,順位
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [58]:
# 差の符号が Minus の順位の和を求める
r_minus = np.sum((diff < 0) * rank)
# 差の符号が Plus の順位の和を求める
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(8, 13)

In [59]:
toy_df['後'] = toy_df['前'] + np.arange(1, 7)
diff = toy_df['後'] - toy_df['前']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['差'] = diff
toy_df['順位'] = rank
toy_df

,前,後,差,順位
0,59,60,1,1
1,52,54,2,2
2,55,58,3,3
3,61,65,4,4
4,59,64,5,5
5,45,51,6,6


In [60]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(0, 21)

In [61]:
toy_df['後'] = toy_df['前'] + [1, -2, -3, 4, 5, -6]
diff = toy_df['後'] - toy_df['前']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['差'] = diff
toy_df['順位'] = rank
toy_df

,前,後,差,順位
0,59,60,1,1
1,52,50,-2,2
2,55,52,-3,3
3,61,65,4,4
4,59,64,5,5
5,45,39,-6,6


In [62]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(11, 10)

In [63]:
# `stats.wilcoxon`関数で仮説検定を行なう
# 対応のある２標本Data を渡す
T, p = stats.wilcoxon(training_rel['前'], training_rel['後'])
print(f'ｐ値: {p:.3f}')

ｐ値: 0.036


In [64]:
# `stats.wilcoxon`関数に対応のある２標本Data の差をわたす
T, p = stats.wilcoxon(training_rel['後'] - training_rel['前'])
print(f'ｐ値: {p:.3f}')

ｐ値: 0.040


帰無仮説は棄却された。

ウィルコクソンの符号付き順位検定は母集団が正規分布に従う場合でも使用できるが、対応ありｔ検定に比べて検出力が下がる。

In [73]:
# 差は N(3, 4**2) に従う Sample size 20 の標本Data を１万組用意しておく
n = 10000
diffs = np.round(stats.norm(3, 4).rvs(size=(n, 20)))

In [74]:
# 対応ありｔ検定
cnt = 0
alpha = 0.05
for diff in diffs:
    t, p = stats.ttest_1samp(diff, 0)
    if p < alpha:
        cnt += 1

cnt / n

0.883

In [78]:
# ウィルコクソンの符号付き順位検定
cnt = 0
alpha = 0.05
for diff in diffs:
    T, p = stats.wilcoxon(diff)
    if p < alpha:
        cnt += 1

cnt / n

0.873

わずかながら「対応ありｔ検定」のほうが検出力が大きいという結果になった。
「母集団が正規分布」に従っている場合は、**対応ありのｔ検定の方が検出力が高い**ということは覚えておく。

## マン・ホイットニーのＵ検定（Mann-Whitney rank test）
- Data に対応がない
- ２標本の母集団に正規分布を仮定できない

場合の中央値の差の検定。

In [65]:
training_ind = pd.read_csv('../data/ch11_training_ind.csv')
toy_df = training_ind[:5].copy()
toy_df

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [66]:
# Ｕ検定では Data 全体に対して値が小さい順に順位付けを行なう
rank = stats.rankdata(np.concatenate([toy_df['A'], toy_df['B']]))
rank_df = pd.DataFrame({'A': rank[:5], 'B': rank[5:10]}).astype(int)
rank_df

,A,B
0,3,5
1,6,8
2,1,9
3,10,4
4,2,7


In [67]:
# U の検定統計量は、 A の順位和を求め、A の大きさを n1 として n1*(N1+1)/2 を引いたもの
n1 = len(rank_df['A'])
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

7.000

n1*(n1+1)/2 は検定統計量の最小値を 0 にするための数
A にいい順位がすべて集まった場合の Data を作って確認してみる

In [68]:
rank_df = pd.DataFrame(np.arange(1, 11).reshape(2, 5).T, columns=['A', 'B'])
rank_df

,A,B
0,1,6
1,2,7
2,3,8
3,4,9
4,5,10


In [69]:
# この場合の検定統計量を計算してみる
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

0.000

0 になることを確認。
逆に A に悪い順位が集まっている場合を確認する。

In [70]:
rank_df = pd.DataFrame(np.arange(1, 11).reshape(2, 5)[::-1].T, columns=['A', 'B'])
rank_df

,A,B
0,6,1
1,7,2
2,8,3
3,9,4
4,10,5


In [71]:
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

25.000

大きい値を確認。
A によい順位が集まっている場合でも悪い順位が集まっている場合でも、２標本の中央値に偏りがあることにはかわりない。
そのためＵ検定は両側検定を行なうことになる。

`scipy.stats` を利用して検定をおこなう

In [72]:
u, p = stats.mannwhitneyu(training_ind['A'], training_ind['B'], alternative='two-sided')
p

0.059

対応なしの ｔ検定と同様に帰無仮説が採択される、という結果になった。
Ｕ検定は母集団が正規分布に従っている場合、対応なしのｔ検定に比べて検出力が低くなる